## Initializing Weather Data

In [9]:
import pandas as pd

### Loading weather & trips

In [10]:
weather_original = pd.read_csv(r'C:\Users\etaiw\Desktop\Projects\aviya_project\Data\Data Weather\2025-02_data_weather.csv')
weather_raw = weather_original

In [11]:
weather_raw.head(5)

,תחנה,תאריך ושעה (שעון חורף),טמפרטורה (C°),טמפרטורת מקסימום (C°),טמפרטורת מינימום (C°),כיוון הרוח (מעלות),מהירות רוח (מטר לשניה),"כמות גשם (מ""מ)"
0,"תל-אביב, חוף",01/02/2025 00:00,18.0,18.1,17.9,290,5.0,0.0
1,"תל-אביב, חוף",01/02/2025 00:10,18.0,18.1,18.0,296,5.1,0.0
2,"תל-אביב, חוף",01/02/2025 00:20,18.0,18.1,18.0,300,5.4,0.0
3,"תל-אביב, חוף",01/02/2025 00:30,18.2,18.3,18.1,316,6.7,0.0
4,"תל-אביב, חוף",01/02/2025 00:40,18.1,18.2,18.0,310,6.6,0.0


### Convert weather data resolution to 30 min intervals

In [12]:
# Hebrew -> English 
col_rename_map = {
    'תחנה': 'station',
    'תאריך ושעה (שעון חורף)': 'datetime',
    'טמפרטורה (C°)': 'temp_c',
    'טמפרטורת מקסימום (C°)': 'temp_max_c',
    'טמפרטורת מינימום (C°)': 'temp_min_c',
    'כיוון הרוח (מעלות)': 'wind_direction_deg',
    'מהירות רוח (מטר לשניה)': 'wind_speed_ms',
    'כמות גשם (מ"מ)': 'rain_mm'
}

# Column rename to English
weather_raw = weather_raw.rename(columns=col_rename_map)
weather_raw.drop(['station'], axis=1, inplace=True)


In [13]:
weather_raw.head(2)

,datetime,temp_c,temp_max_c,temp_min_c,wind_direction_deg,wind_speed_ms,rain_mm
0,01/02/2025 00:00,18.0,18.1,17.9,290,5.0,0.0
1,01/02/2025 00:10,18.0,18.1,18.0,296,5.1,0.0


In [14]:
import pandas as pd

# Convert the 'datetime' column from string to a proper datetime object
weather_raw['datetime'] = pd.to_datetime(weather_raw['datetime'], dayfirst=True)

# Create a new column 'datetime_30' that rounds down each timestamp
# to the start of a 30-minute interval (00:00, 00:30, 01:00, etc.).
weather_raw['datetime_30'] = weather_raw['datetime'].dt.floor('30T')  # '30T' = 30 minutes

# Aggregate the data to 30-minute resolution:
#  - Group by 'datetime_30' (each 30-minute window)
#  - For each group of 3 rows (10-min steps):
#        * temp_c:      average temperature
#        * temp_max_c:  maximum temperature
#        * temp_min_c:  minimum temperature
#        * wind_direction_deg: average wind direction
#        * wind_speed_ms:      average wind speed
#        * rain_mm:            sum of rain in that 30-minute window
weather = (
    weather_raw
    .groupby('datetime_30')
    .agg(
        temp_c=('temp_c', 'mean'),                     # average temperature over 30 minutes
        temp_max_c=('temp_max_c', 'max'),              # highest temp in the 30-min window
        temp_min_c=('temp_min_c', 'min'),              # lowest temp in the 30-min window
        wind_direction_deg=('wind_direction_deg', 'mean'),  # average wind direction
        wind_speed_ms=('wind_speed_ms', 'mean'),       # average wind speed
        rain_mm=('rain_mm', 'sum')                     # total rain in 30 minutes
    )
    .reset_index()
    .rename(columns={'datetime_30': 'datetime'})       # rename back to 'datetime' for clarity
)

# Quick check: show the first 5 rows of the 30-minute resolution dataset
print(weather.head())


             datetime     temp_c  temp_max_c  temp_min_c  wind_direction_deg  \
0 2025-02-01 00:00:00  18.000000        18.1        17.9          295.333333   
1 2025-02-01 00:30:00  18.066667        18.3        17.9          312.333333   
2 2025-02-01 01:00:00  17.866667        17.9        17.8          321.333333   
3 2025-02-01 01:30:00  17.866667        18.0        17.8          328.333333   
4 2025-02-01 02:00:00  17.766667        17.8        17.7          327.666667   

   wind_speed_ms  rain_mm  
0       5.166667      0.0  
1       6.433333      0.0  
2       5.500000      0.0  
3       6.133333      0.0  
4       5.733333      0.0  


C:\Users\etaiw\AppData\Local\Temp\ipykernel_5464\3663084359.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  weather_raw['datetime_30'] = weather_raw['datetime'].dt.floor('30T')  # '30T' = 30 minutes


In [15]:
# save as csv to clean data 
weather.to_csv(r'C:\Users\etaiw\Desktop\Projects\aviya_project\Data\Clean Data\Final Weather\2025-02_Final_Weather.csv', index=False)